In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import sys


In [ ]:
IMAGE_WIDTH = 512
IMAGE_HEIGHT = 512
BATCH_SIZE = 32
NUM_EPOCHS = 100
NUM_CLASSES = 4


In [ ]:
def loadDataset(datagen, path):
    dataset = datagen.flow_from_directory(
        directory=path,
        target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
        color_mode="rgb",  # Change to RGB for 3 channels
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        classes=["glioma_tumor", "meningioma_tumor","no_tumor","pituitary_tumor"],
    )
    return dataset

In [ ]:
def ConstructModel():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    base_model.trainable = False  # Freeze the VGG16 base
    
    model = Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.BatchNormalization(),
        layers.Dense(128, activation="relu"),
        layers.BatchNormalization(),  
        layers.Dropout(0.2),
        layers.Dense(64, activation="relu"),
        layers.BatchNormalization(),  
        layers.Dropout(0.2),
        layers.Dense(32, activation="relu"),
        layers.BatchNormalization(),  
        layers.Dense(16, activation="relu"),
        layers.BatchNormalization(),  
        layers.Dense(8, activation="relu"),
        layers.BatchNormalization(),  
        layers.Dense(NUM_CLASSES, activation="sigmoid"),
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy", Precision(), Recall()],
    )

    model.summary()
    return model  

def trainModel(model, training_data_generator, testing_data_generator):
    train_dataset = loadDataset(training_data_generator, r"C:\Users\BME\Documents\BMEG3105\Brain-Tumor-Classification-DataSet\Training")
    validation_dataset = loadDataset(training_data_generator, r"C:\Users\BME\Documents\BMEG3105\Brain-Tumor-Classification-DataSet\Validation")
    history = model.fit(
        train_dataset,
        epochs=NUM_EPOCHS,
        validation_data=validation_dataset,
        callbacks=[EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True), ModelCheckpoint('model_best.keras', monitor='val_loss', save_best_only=True, verbose=1)]
    )
    plot_metrics(history)

def testModel(model, testing_data_generator):
    test_dataset = loadDataset(testing_data_generator, r"C:\Users\BME\Documents\BMEG3105\Brain-Tumor-Classification-DataSet\Testing")
    predictions = model.predict(test_dataset)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_dataset.classes
    loss, accuracy, precision, recall = model.evaluate(test_dataset)
    confusion = confusion_matrix(true_classes, predicted_classes)
    print(f"Testing loss: {loss}")
    print(f"Testing accuracy: {accuracy}")
    print(f"Testing precision: {precision}")
    print(f"Testing recall: {recall}")
    print(f"Testing confusion matrix:")
    print(confusion)

In [ ]:
def plot_metrics(history):
    metrics = ['loss', 'accuracy', 'precision', 'recall']
    for metric in metrics:
        plt.figure(figsize=(8, 6))
        plt.plot(history.history[f'{metric}'], label=f'Train {metric.capitalize()}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.tight_layout()
        plt.savefig(f'{metric}_metric.png')
        plt.show()

def save_model_plot(model, filename="model.png"):
    # plot_model(model, to_file=filename, show_shapes=True, show_layer_names=True)
    print(f"Model architecture diagram saved as {filename}")

In [ ]:
def main():
    model = ConstructModel()
    training_data_generator = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        brightness_range=[0.8, 1.2],
        channel_shift_range=0.2
    )
    testing_data_generator = ImageDataGenerator(
        rescale=1.0 / 255
    )
    trainModel(model, training_data_generator, testing_data_generator)
    print("finish training")
    model.save("tumour_exist classification.keras")
    print("finish save the model")
    print("test model now ")
    testModel(model, testing_data_generator)

In [ ]:
if __name__ == "__main__":
    main()